In [ ]:
import pandas as pd
import cufflinks as cf
import warnings
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chisquare
from  functools import reduce
import re
from plotly.offline import plot,iplot
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px#graficos express
import sys
warnings.filterwarnings("ignore")
cf.go_offline()
pd.set_option("display.max_columns",200)

In [ ]:
def chi_square(df,col,valor_miss):
    x_i=df[col].fillna(valor_miss).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        print("Rechazamos HO(La porporción de categorias es la misma que la general)")
    else:
        print("Aceptamos HO(La porporción de categorias es la misma que la general)")

In [ ]:
def other(df,col,n):
    aux=list(df[col].value_counts(1).index[n:].values)
    dictio_aux=dict(zip(aux,["otro"]*len(aux)))
    df[col]=df[col].replace(dictio_aux)
    return df

def multiple_replace(string, rep_dict):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(rep_dict,key=len,reverse=True)]), flags=re.DOTALL)
    return pattern.sub(lambda x: rep_dict[x.group(0)]+"_", string)

In [ ]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=True):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
        dictionary: Dictionary of column names (keys) and GVIF ** (1 / (2 * Df)) ** 2 (values)
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final,gvif_filter,df_m


In [ ]:
sys.path.append("../libreria/")

In [ ]:
from libreria import rename_cols,completitud,text_clean,box,histogram

In [ ]:
df=pd.read_csv("../data/visualizaciones/dataset_house_prices.csv")

## Variables

**El problema**

En análisis exploratorio tiene el objetivo de entender el significado y la relevancia de cada variable con respecto al problema. Además apartir de este análisis se pueden determinar que acciones tomar para cada variable En este caso:

- La relevancia de la variable en la compra de una casa.
- La importancia de la variable.
- Solapamiento con otras variables.


Variable objetivo : 'SalePrice'.

- MSSubClass: clase de construcción
- MSZoning: clasificación de la zona
- LotFrontage: pies lineales de calle de la parcela
- LotArea: tamaño de la parcela en pies cuadrados
- Street: tipo de acceso por carretera
- Alley: tipo de acceso al callejón
- LotShape: forma de la parcela
- LandContour: planitud de la parcela
- Utilities: servicios públicos disponibles
- LotConfig: Configuración de parcela
- LandSlope: pendiente de la parcela
- Neighborhood: ubicación física dentro de los límites de la ciudad de Ames
- Condition1: proximidad a la carretera principal o al ferrocarril
- Condition2: proximidad a la carretera principal o al ferrocarril (si hay un segundo)
- BldgType: tipo de vivienda
- HouseStyle: estilo de vivienda
- OverallQual: calidad general del material y del acabado
- OverallCond: condición general
- YearBuilt: fecha original de construcción
- YearRemodAdd: fecha de remodelación
- RoofStyle: tipo de cubierta
- RoofMatl: material del techo
- Exterior1st: revestimiento exterior de la casa
- Exterior2nd: revestimiento exterior de la casa (si hay más de un material)
- MasVnrType: tipo de revestimiento de mampostería
- MasVnrArea: área de revestimiento de mampostería en pies cuadrados
- ExterQual: calidad del material exterior
- ExterCond: estado del material en el exterior
- Foundation: tipo de cimentación
- BsmtQual: altura del sótano
- BsmtCond: estado general del sótano
- BsmtExposure: paredes del sótano a nivel de calle o de jardín
- BsmtFinType1: calidad del área acabada del sótano
- BsmtFinSF1: pies cuadrados de la superficie acabada tipo 1
- BsmtFinType2: calidad de la segunda superficie acabada (si existe)
- BsmtFinSF2: Pies cuadrados de la superficie acabada tipo 2
- BsmtUnfSF: pies cuadrados del área sin terminar del sótano
- TotalBsmtSF: pies cuadrados totales del sótano
- Heating: tipo de calefacción
- HeatingQC: calidad y estado de la calefacción
- CentralAir: aire acondicionado central
- Electrical: sistema eléctrico
- 1erFlrSF: área en pies cuadrados de la primera planta (o planta baja)
- 2ndFlrSF: área en pies cuadrados de la segunda planta
- LowQualFinSF: pies cuadrados acabados de baja calidad (todos los pisos)
- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- BsmtFullBath: cuartos de baño completos en el sótano
- BsmtHalfBath: medio baño del sótano
- FullBath: baños completos sobre el nivel del suelo
- HalfBath: medios baños sobre el nivel del suelo
- Bedroom: número de dormitorios por encima del nivel del sótano
- Kitchen: número de cocinas
- KitchenQual: calidad de la cocina
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- Functional: valoración de la funcionalidad de la vivienda
- Fireplaces: número de chimeneas
- FireplaceQu: calidad de la chimenea
- GarageType: ubicación del garaje
- GarageYrBlt: año de construcción del garaje
- GarageFinish: acabado interior del garaje
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados
- GarageQual: calidad de garaje
- GarageCond: condición de garaje
- PavedDrive: calzada asfaltada
- WoodDeckSF: area de plataforma de madera en pies cuadrados
- OpenPorchSF: área de porche abierto en pies cuadrados
- EnclosedPorch: área de porche cerrada en pies cuadrados
- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados
- PoolQC: calidad de la piscina
- Fence: calidad de la valla
- MiscFeature: característica miscelánea no cubierta en otras categorías
- MiscVal: valor en dólares de la característica miscelánea
- MoSold: mes de venta
- YrSold: año de venta
- SaleType: tipo de venta
- SaleCondition: Condiciones de venta

In [ ]:
v_feats=["MSSubClass","MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Condition1","Condition2","BldgType","HouseStyle","OverallQual","OverallCond",
        "YearBuilt","YearRemodAdd","RoofStyle","RoofMatl","MasVnrType","ExterQual",
        "ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
        "Heating","HeatingQC","CentralAir","Electrical","LowQualFinSF","BsmtHalfBath","FullBath","KitchenAbvGr","KitchenQual","Functional","Fireplaces","FireplaceQu",
        "GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence",
        "MiscFeature","MoSold","YrSold","SaleType","SaleCondition"]
c_feats=["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF",
        "2ndFlrSF","GrLivArea","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch",
        "PoolArea","MiscVal","BedroomAbvGr","HalfBath","GarageCars","BsmtFullBath","TotRmsAbvGrd"]
tgt=["SalePrice"]

df=rename_cols(df,v_feats,"v_")
df=rename_cols(df,c_feats,"c_")
df=rename_cols(df,tgt,"tgt_")

In [ ]:
#Eliminamos aquellas variables que que hacen referencia a la venta
df.drop(columns=["v_SaleType","v_SaleCondition"],inplace=True)

In [ ]:
df.drop(columns=["Neighborhood","Exterior2nd","Exterior1st","v_YrSold","v_MoSold"],inplace=True)

In [ ]:
#NO hay duplicados
df[df.duplicated(subset=["Id"])]

,Id,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_Alley,v_LotShape,v_LandContour,v_Utilities,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_FireplaceQu,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,v_PoolQC,v_Fence,v_MiscFeature,c_MiscVal,tgt_SalePrice


In [ ]:
#Variables para limpiar el texto
features=list(filter(lambda x: x not in ["v_GarageYrBlt","v_YearBuilt"],df.filter(like="v_") ))


for col in features:
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)

In [ ]:
histogram(df,"tgt_SalePrice",5,"tgt_SalePrice").show()

In [ ]:
miss=completitud(df)

In [ ]:
miss

,columna,total,completitud
0,v_PoolQC,1453,0.479452
1,v_MiscFeature,1406,3.698630
2,v_Alley,1369,6.232877
3,v_Fence,1179,19.246575
4,v_FireplaceQu,690,52.739726
...,...,...,...
69,c_MiscVal,0,100.000000
70,v_HeatingQC,0,100.000000
71,v_CentralAir,0,100.000000
72,c_2ndFlrSF,0,100.000000


In [ ]:
null_cols=list(miss[miss["completitud"]<80]["columna"].values)

In [ ]:
null_cols

['v_PoolQC', 'v_MiscFeature', 'v_Alley', 'v_Fence', 'v_FireplaceQu']

In [ ]:
df=df.drop(columns=null_cols)

In [ ]:
df["v_MSSubClass"]=df["v_MSSubClass"].astype(int)

In [ ]:
df_multi=df.copy()

In [ ]:
df_multi=df.copy()

## Isolation Forest

**La clase sklearn.ensemble.IsolationForest incorpora las principales funcionalidades que se necesitan a la hora de trabajar con modelos Isolation Forest. Los principales argumentos para entrenar este tipo de modelos son:**

- n_estimators: número de árboles que forman el modelo.

- max_samples: número de observaciones empleadas para entrenar cada árbol.

- max_features : El número de características que se extraerán de X para entrenar cada estimador base.

- contamination: proporción de anomalías esperadas en los datos de entrenamiento. En base a este valor, se establece el límite acorde al cual se clasifican las observaciones en normales o anómalas.

- random_state: semilla para garantizar la reproducibilidad de los resultados.

- Se procede a entrenar un modelo asumiendo que hay un 1% de observaciones anómalas en el conjunto de entrenamiento.

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
df=df.drop(columns=["Id"])

In [ ]:
df.shape

(1460, 68)

In [ ]:
df_iso=df.filter(like="c_").dropna()

In [ ]:
#max_features=1
max_features=df_iso.shape[1]
n_estimators=50
max_samples='auto'
contamination=float(0.2)
model=IsolationForest(max_features = max_features, n_estimators=n_estimators, max_samples=max_samples, contamination=contamination)
model.fit(df_iso)

IsolationForest(contamination=0.2, max_features=23, n_estimators=50)

- Los modelos IsolationForest tienen dos métodos de predicción con los que se obtiene distinta información.

- Con el método predict() se devuelve directamente la clasificación de anomalía (-1) o no anomalía (1) acorde a la proporción de contaminación que se ha indicado en la definición del modelo.

In [ ]:
df_iso["outlier"]=model.predict(df_iso)

In [ ]:
df_iso["outlier"].value_counts()

 1    956
-1    239
Name: outlier, dtype: int64

In [ ]:
df_iso[df_iso["outlier"]==-1].shape[0]/df_iso.shape[0]

0.2

In [ ]:
indices=list(df_iso[df_iso["outlier"]==1].index)

In [ ]:
df=df.iloc[indices]

In [ ]:
df.reset_index(drop=True,inplace=True)

## MISSINGS

### Variables unitarias

In [ ]:
#Eliminamos Variables unarias

In [ ]:
df[["v_Utilities","v_Street"]].describe()

,v_Utilities,v_Street
count,956,956
unique,1,2
top,allpub,pave
freq,956,954


In [ ]:
df.drop(columns=["v_Utilities"],inplace=True)

In [ ]:
miss=completitud(df)

miss_cols=list(miss[miss["completitud"]<100]["columna"].values)

In [ ]:
miss

,columna,total,completitud
0,v_GarageQual,44,95.39749
1,v_GarageType,44,95.39749
2,v_GarageYrBlt,44,95.39749
3,v_GarageFinish,44,95.39749
4,v_GarageCond,44,95.39749
...,...,...,...
62,c_BsmtUnfSF,0,100.00000
63,c_TotalBsmtSF,0,100.00000
64,c_MiscVal,0,100.00000
65,v_YearBuilt,0,100.00000


In [ ]:
miss_cols

['v_GarageQual',
 'v_GarageType',
 'v_GarageYrBlt',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtFinType2',
 'v_BsmtQual',
 'v_BsmtFinType1',
 'v_BsmtCond',
 'v_Electrical']

### NORMALIZAMOS DISCRETAS

In [ ]:
v_=list(filter(lambda x:"v_" in x,miss_cols))
v_=[x for x in v_ if x!="v_GarageYrBlt"]

In [ ]:
for col in v_:
    print(col)
    print(df[col].isnull().sum())
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)
    print(df[col].isnull().sum())

v_GarageQual
44
44
v_GarageType
44
44
v_GarageFinish
44
44
v_GarageCond
44
44
v_BsmtExposure
23
23
v_BsmtFinType2
22
22
v_BsmtQual
22
22
v_BsmtFinType1
22
22
v_BsmtCond
22
22
v_Electrical
1
1


In [ ]:
for col in v_:
    print(col)
    display(df[col].value_counts(1))
    print("\n")

v_GarageQual


ta    0.938596
fa    0.046053
gd    0.009868
po    0.003289
ex    0.002193
Name: v_GarageQual, dtype: float64



v_GarageType


attchd     0.600877
detchd     0.330044
builtin    0.047149
basment    0.014254
carport    0.005482
2types     0.002193
Name: v_GarageType, dtype: float64



v_GarageFinish


...

unf    0.353319
glq    0.253747
alq    0.155246
blq    0.099572
rec    0.095289
lwq    0.042827
Name: v_BsmtFinType1, dtype: float64



v_BsmtCond


ta    0.919700
gd    0.042827
fa    0.036403
po    0.001071
Name: v_BsmtCond, dtype: float64



v_Electrical


sbrkr    0.895288
fusea    0.077487
fusef    0.023037
fusep    0.003141
mix      0.001047
Name: v_Electrical, dtype: float64

In [ ]:
##"v_GarageType"
##"v_Electrical"
#MasVnrType

In [ ]:
dictionary={"v_BldgType":1,"v_Foundation":2,"v_Functional":1,"v_Heating":1,"v_HouseStyle":3,"v_LandContour":1,
"v_LandSlope":1,"v_LotConfig":2,"v_LotShape":1,"v_MSZoning":2,"v_RoofMatl":1,"v_RoofStyle":2,"v_Condition2":1,"v_Condition1":1,"v_BsmtFinType2":1,"v_BsmtFinType1":3,"v_BsmtExposure":1}

In [ ]:
for i,j in dictionary.items():
    print(i)
    df=other(df,i,j)
    print(df[i].value_counts())

v_BldgType
1fam    806
otro    150
Name: v_BldgType, dtype: int64
v_Foundation
pconc     422
cblock    401
otro      133
Name: v_Foundation, dtype: int64
v_Functional
typ     899
otro     57
Name: v_Functional, dtype: int64
v_Heating
gasa    935
otro     21
Name: v_Heating, dtype: int64
v_HouseStyle
1story    493
2story    266
1 5fin    116
otro       81
Name: v_HouseStyle, dtype: int64
v_LandContour
lvl     869
otro     87
Name: v_LandContour, dtype: int64
v_LandSlope
gtl     923
otro     33
Name: v_LandSlope, dtype: int64
v_LotConfig
inside    736
corner    159
otro       61
Name: v_LotConfig, dtype: int64
v_LotShape
reg     698
otro    258
Name: v_LotShape, dtype: int64
v_MSZoning
rl      731
rm      157
otro     68
Name: v_MSZoning, dtype: int64
v_RoofMatl
compshg    948
otro         8
Name: v_RoofMatl, dtype: int64
v_RoofStyle
gable    785
hip      155
otro      16
Name: v_RoofStyle, dtype: int64
v_Condition2
norm    947
otro      9
Name: v_Condition2, dtype: int64
v_Condition1
no

In [ ]:
dictio_GarageType=dict(zip(list(df["v_GarageType"].value_counts().index[-3:]),["otro"]*3))
dictio_electrical=dict(zip(list(df["v_Electrical"].value_counts().index[-3:]),["otro"]*3))
dictio_MasVnrType=dict(zip(list(df["v_MasVnrType"].value_counts().index[-2:]),["otro"]*2))

In [ ]:
df["v_GarageType"]=df["v_GarageType"].replace(dictio_GarageType)
df["v_Electrical"]=df["v_Electrical"].replace(dictio_electrical)
df["v_MasVnrType"]=df["v_MasVnrType"].replace(dictio_MasVnrType)

In [ ]:
df["v_GarageType"].value_counts(1)

attchd     0.600877
detchd     0.330044
builtin    0.047149
otro       0.021930
Name: v_GarageType, dtype: float64

In [ ]:
df["v_Electrical"].value_counts(1)

sbrkr    0.895288
fusea    0.077487
otro     0.027225
Name: v_Electrical, dtype: float64

In [ ]:
## "v_GarageQual","v_GarageCond"

In [ ]:
df["v_GarageQual"].value_counts(1)

ta    0.938596
fa    0.046053
gd    0.009868
po    0.003289
ex    0.002193
Name: v_GarageQual, dtype: float64

In [ ]:
df["v_GarageCond"].value_counts(1)

ta    0.952851
fa    0.032895
po    0.006579
gd    0.005482
ex    0.002193
Name: v_GarageCond, dtype: float64

In [ ]:
dictio_qual_cond=dict(zip(["ex","gd","ta","fa","po"],[3,3,2,1,1]))

In [ ]:
feats_replace_cond_qual=["v_GarageQual","v_GarageCond","v_BsmtCond","v_BsmtQual","v_ExterCond","v_ExterQual","v_HeatingQC","v_KitchenQual"]

In [ ]:
for feat in feats_replace_cond_qual:
    print(feat)
    df[feat]=df[feat].replace(dictio_qual_cond)
    print(df[feat].value_counts(1))
    print("\n")

v_GarageQual
2.0    0.938596
1.0    0.049342
3.0    0.012061
Name: v_GarageQual, dtype: float64


v_GarageCond
2.0    0.952851
1.0    0.039474
3.0    0.007675
Name: v_GarageCond, dtype: float64


v_BsmtCond
2.0    0.919700
3.0    0.042827
1.0    0.037473
Name: v_BsmtCond, dtype: float64


v_BsmtQual
3.0    0.488223
2.0    0.478587
1.0    0.033191
Name: v_BsmtQual, dtype: float64


v_ExterCond
2    0.881799
3    0.094142
1    0.024059
Name: v_ExterCond, dtype: float64


v_ExterQual
2    0.651674
3    0.341004
1    0.007322
Name: v_ExterQual, dtype: float64


v_HeatingQC
3    0.671548
2    0.293933
1    0.034519
Name: v_HeatingQC, dtype: float64


v_KitchenQual
2    0.528243
3    0.441423
1    0.030335
Name: v_KitchenQual, dtype: float64




In [ ]:
miss=completitud(df)
miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,44,95.397490
1,v_GarageType,44,95.397490
2,v_GarageYrBlt,44,95.397490
3,v_GarageFinish,44,95.397490
4,v_GarageCond,44,95.397490
5,v_BsmtExposure,23,97.594142
6,v_BsmtFinType2,22,97.698745
7,v_BsmtQual,22,97.698745
8,v_BsmtFinType1,22,97.698745
9,v_BsmtCond,22,97.698745


In [ ]:
#Remplazamos los valores de la fecha de construccion del garage por la fecha de construcción de la propiedad
df['v_GarageYrBlt']=df['v_GarageYrBlt'].fillna(df['v_YearBuilt']).astype(float)

In [ ]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,44,95.397490
1,v_GarageType,44,95.397490
2,v_GarageFinish,44,95.397490
3,v_GarageCond,44,95.397490
4,v_BsmtExposure,23,97.594142
5,v_BsmtQual,22,97.698745
6,v_BsmtFinType2,22,97.698745
7,v_BsmtCond,22,97.698745
8,v_BsmtFinType1,22,97.698745
9,v_Electrical,1,99.895397


### CATEGORICAS

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test=train_test_split(df,test_size=.2,random_state=413)

In [ ]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,44,95.397490
1,v_GarageType,44,95.397490
2,v_GarageFinish,44,95.397490
3,v_GarageCond,44,95.397490
4,v_BsmtExposure,23,97.594142
5,v_BsmtQual,22,97.698745
6,v_BsmtFinType2,22,97.698745
7,v_BsmtCond,22,97.698745
8,v_BsmtFinType1,22,97.698745
9,v_Electrical,1,99.895397


In [ ]:
v_feats=list(filter(lambda x:"v_" in x,list(miss[miss["completitud"]<100]["columna"].values)))

In [ ]:
v_feats

['v_GarageQual',
 'v_GarageType',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtQual',
 'v_BsmtFinType2',
 'v_BsmtCond',
 'v_BsmtFinType1',
 'v_Electrical']

In [ ]:
for column in v_feats:
    print(column)
    value=X_train[column].value_counts().index[0]
    print(value)
    print(chi_square(df,column,value))
    print("\n")

v_GarageQual
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageType
attchd
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageFinish
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtExposure
no
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtQual
3.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType2
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType1
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_Electrical
sbrkr
Aceptamos HO(La porporción de categorias es la misma que la general)
None




In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp=SimpleImputer(missing_values=np.nan,strategy="most_frequent")

In [ ]:
imp.fit(X_train[v_feats])

SimpleImputer(strategy='most_frequent')

In [ ]:
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

In [ ]:
X_train[v_feats]=pd.DataFrame(imp.transform(X_train[v_feats]),columns=v_feats)

In [ ]:
X_test[v_feats]=pd.DataFrame(imp.transform(X_test[v_feats]),columns=v_feats)

In [ ]:
miss=completitud(X_train)

miss[miss["completitud"]<100]

,columna,total,completitud


In [ ]:
miss=completitud(X_test)

miss[miss["completitud"]<100]

,columna,total,completitud


## Reduccion de dimensiones

In [ ]:
X_train["v_GarageYrBlt"]=X_train["v_GarageYrBlt"].astype(int)
X_test["v_GarageYrBlt"]=X_test["v_GarageYrBlt"].astype(int)
X_train["v_YearRemodAdd"]=X_train["v_YearRemodAdd"].astype(int)
X_test["v_YearRemodAdd"]=X_test["v_YearRemodAdd"].astype(int)

In [ ]:
X_train

,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal,tgt_SalePrice
0,20,otro,84.0,11096,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,8,5,2006,2006,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,otro,glq,24,unf,0,1632,1656,gasa,3,y,sbrkr,1656,0,0,1656,0,0,2,0,3,1,3,7,typ,0,attchd,2006,rfn,3,826,2.0,2.0,y,0,146,0,0,0,0,0,249700
1,120,rl,61.0,7658,pave,reg,lvl,inside,gtl,norm,norm,otro,1story,9,5,2005,2005,hip,compshg,brkface,412.0,3,2,pconc,3.0,2.0,no,glq,456,unf,0,1296,1752,gasa,3,y,sbrkr,1752,0,0,1752,1,0,2,0,2,1,3,6,typ,1,attchd,2005,rfn,2,576,2.0,2.0,y,196,82,0,0,0,0,0,319900
2,20,rl,60.0,9600,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,7,1950,1995,gable,compshg,none,0.0,2,2,cblock,3.0,2.0,no,otro,442,unf,0,625,1067,gasa,2,y,sbrkr,1067,0,0,1067,0,0,2,0,2,1,3,4,otro,0,attchd,1996,unf,2,436,2.0,2.0,y,290,0,0,0,0,0,0,128000
3,30,rl,80.0,11600,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,5,1922,1950,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0,unf,0,700,700,gasa,3,y,sbrkr,1180,0,0,1180,0,0,1,0,2,1,1,5,typ,1,detchd,1922,unf,1,252,2.0,1.0,y,0,0,67,0,0,0,0,137500
4,30,rl,60.0,5400,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,5,6,1920,1950,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0,unf,0,691,691,gasa,3,y,fusea,691,0,0,691,0,0,1,0,2,1,3,4,typ,0,detchd,1920,unf,1,216,1.0,2.0,n,0,20,94,0,0,0,0,86000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,20,rl,60.0,11664,pave,reg,lvl,inside,gtl,otro,norm,1fam,1story,6,5,1948,1950,gable,compshg,brkface,206.0,2,2,cblock,2.0,1.0,no,otro,336,unf,0,746,1082,gasa,2,y,sbrkr,1082,0,0,1082,0,0,1,0,2,1,2,5,typ,1,detchd,1948,unf,1,240,2.0,2.0,y,0,130,0,0,0,0,0,119200
760,20,rl,60.0,7024,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,5,2005,2006,gable,compshg,none,0.0,2,2,pconc,3.0,3.0,no,alq,980,unf,0,110,1090,gasa,3,y,sbrkr,1090,0,0,1090,1,0,1,1,2,1,2,5,typ,0,attchd,2005,fin,2,450,2.0,2.0,y,0,49,0,0,0,0,0,157000
761,50,rl,152.0,12134,pave,otro,otro,inside,otro,norm,norm,1fam,1 5fin,8,7,1988,2005,gable,compshg,none,0.0,3,2,otro,3.0,2.0,otro,glq,427,unf,0,132,559,gasa,3,y,sbrkr,1080,672,0,1752,0,0,2,0,4,1,2,8,typ,0,otro,1988,rfn,2,492,2.0,2.0,y,325,12,0,0,0,0,0,164000
762,190,rl,107.0,10615,pave,otro,otro,corner,otro,otro,otro,otro,2story,3,5,1900,1970,gable,compshg,none,0.0,2,2,cblock,1.0,2.0,otro,otro,440,unf,0,538,978,gasa,2,y,sbrkr,1014,685,0,1699,1,0,2,0,3,2,2,7,typ,0,otro,1920,unf,2,420,1.0,1.0,y,0,74,0,0,0,0,0,95000


### Relación de valor perdido

In [ ]:
completitud(X_train)

,columna,total,completitud
0,v_MSSubClass,0,100.0
1,v_CentralAir,0,100.0
2,v_Electrical,0,100.0
3,c_1stFlrSF,0,100.0
4,c_2ndFlrSF,0,100.0
...,...,...,...
62,c_BsmtFinSF1,0,100.0
63,v_BsmtFinType2,0,100.0
64,c_BsmtFinSF2,0,100.0
65,v_YearRemodAdd,0,100.0


### Filtro de baja varianza

In [ ]:
X_train.filter(like="c_").describe(percentiles=np.arange(0.1,1.1,.1))

,c_LotFrontage,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,c_1stFlrSF,c_2ndFlrSF,c_GrLivArea,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_TotRmsAbvGrd,c_GarageCars,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal
count,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.0,764.000000
mean,68.428010,9179.824607,76.850785,368.713351,19.782723,612.836387,1001.332461,1095.595550,302.108639,1401.861257,0.335079,0.321990,2.812827,6.293194,1.714660,450.692408,72.854712,38.361257,19.353403,2.155759,10.873037,0.0,12.178010
std,21.312843,4135.616395,134.998456,380.668014,87.050203,426.012205,354.327221,314.804701,388.490684,392.605427,0.472327,0.467545,0.690921,1.388175,0.696175,192.051246,101.603351,51.205626,54.077012,22.599031,45.639287,0.0,107.467644
min,21.000000,1300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,438.000000,0.000000,438.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10%,43.000000,4595.400000,0.000000,0.000000,0.000000,112.600000,630.000000,734.300000,0.000000,894.000000,0.000000,0.000000,2.000000,5.000000,1.000000,240.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
20%,53.000000,6762.000000,0.000000,0.000000,0.000000,219.800000,741.600000,832.000000,0.000000,1040.000000,0.000000,0.000000,2.000000,5.000000,1.000000,288.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
30%,60.000000,7835.200000,0.000000,0.000000,0.000000,321.900000,830.700000,892.000000,0.000000,1154.900000,0.000000,0.000000,3.000000,6.000000,1.000000,352.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
40%,61.200000,8499.200000,0.000000,24.000000,0.000000,426.000000,885.600000,960.200000,0.000000,1264.200000,0.000000,0.000000,3.000000,6.000000,2.000000,424.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,68.000000,9049.000000,0.000000,320.500000,0.000000,553.000000,954.500000,1050.000000,0.000000,1390.000000,0.000000,0.000000,3.000000,6.000000,2.000000,461.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.0,0.000000
60%,72.000000,9600.000000,0.000000,454.800000,0.000000,677.000000,1040.800000,1128.000000,293.600000,1501.000000,0.000000,0.000000,3.000000,7.000000,2.000000,492.800000,49.000000,36.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [ ]:
X_train.drop(columns=["c_BsmtFullBath","c_PoolArea"],inplace=True)
X_test.drop(columns=["c_BsmtFullBath","c_PoolArea"],inplace=True)

In [ ]:
X_train.drop(columns=["c_MasVnrArea"],inplace=True)
X_test.drop(columns=["c_MasVnrArea"],inplace=True)

### Filtro de alta correlación

- La Correlación se utiliza para probar las relaciones entre variables cuantitativas o variables categóricas. En otras palabras, es una medida de cómo se relacionan las cosas. El estudio de cómo se correlacionan las variables se denomina análisis de correlación.
- Un coeficiente de correlación proporciona un resumen numérico del grado de asociación entre dos variables.
- Tipos:
- **Correlación r de Pearson: la correlación r de Pearson es el estadístico de correlación más utilizado para medir el grado de relación entre variables relacionadas linealmente. Por ejemplo, en el mercado de valores, si queremos medir cómo se relacionan dos acciones entre sí, se utiliza la correlación r de Pearson para medir el grado de relación entre las dos. La correlación punto-biserial se realiza con la fórmula de correlación de Pearson excepto que una de las variables es dicotómica.**
- **Correlación de rango de Kendall: la correlación de rango de Kendall es una prueba no paramétrica que mide la fuerza de la dependencia entre dos variables. Si consideramos dos muestras, ayb, donde cada tamaño de muestra es n, sabemos que el número total de emparejamientos con a b es n (n-1) / 2.**
- **Correlación de rango de Spearman: la correlación de rango de Spearman es una prueba no paramétrica que se utiliza para medir el grado de asociación entre dos variables. La prueba de correlación de rango de Spearman no conlleva ninguna suposición sobre la distribución de los datos y es el análisis de correlación apropiado cuando las variables se miden en una escala que es al menos ordinal.**

https://www.google.com/url?sa=i&url=https%3A%2F%2Fmedium.com%2F%40outside2SDs%2Fan-overview-of-correlation-measures-between-categorical-and-continuous-variables-4c7f85610365&psig=AOvVaw3KaiD8_vD816QPDiI7Uz8s&ust=1622782900559000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCIjvg8fX-vACFQAAAAAdAAAAABAD

In [ ]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados

In [ ]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")

In [ ]:
X_train.filter(like="v_").columns

Index(['v_MSSubClass', 'v_MSZoning', 'v_Street', 'v_LotShape', 'v_LandContour',
       'v_LotConfig', 'v_LandSlope', 'v_Condition1', 'v_Condition2',
       'v_BldgType', 'v_HouseStyle', 'v_OverallQual', 'v_OverallCond',
       'v_YearBuilt', 'v_YearRemodAdd', 'v_RoofStyle', 'v_RoofMatl',
       'v_MasVnrType', 'v_ExterQual', 'v_ExterCond', 'v_Foundation',
       'v_BsmtQual', 'v_BsmtCond', 'v_BsmtExposure', 'v_BsmtFinType1',
       'v_BsmtFinType2', 'v_Heating', 'v_HeatingQC', 'v_CentralAir',
       'v_Electrical', 'v_LowQualFinSF', 'v_BsmtHalfBath', 'v_FullBath',
       'v_KitchenAbvGr', 'v_KitchenQual', 'v_Functional', 'v_Fireplaces',
       'v_GarageType', 'v_GarageYrBlt', 'v_GarageFinish', 'v_GarageQual',
       'v_GarageCond', 'v_PavedDrive'],
      dtype='object')

In [ ]:
corr=abs(corr)

In [ ]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


,c_BsmtUnfSF
c_BsmtUnfSF,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [ ]:
X_train.drop(columns=["c_TotRmsAbvGrd","c_GarageCars"],inplace=True)
X_test.drop(columns=["c_TotRmsAbvGrd","c_GarageCars"],inplace=True)

In [ ]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")
corr=abs(corr)

In [ ]:
X_train.drop(columns=["c_1stFlrSF"],inplace=True)
X_test.drop(columns=["c_1stFlrSF"],inplace=True)

In [ ]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


,c_BsmtUnfSF
c_BsmtUnfSF,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [ ]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

### Correlación (con el objetivo)

In [ ]:
feats=list(X_train.filter(like="c_").columns)+["tgt_SalePrice"]

In [ ]:
X_train[feats].corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

In [ ]:
corr=X_train[feats].corr(method="spearman")

In [ ]:
corr=abs(corr)

In [ ]:
corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False).style.background_gradient()

,tgt_SalePrice
tgt_SalePrice,1.000000
c_GrLivArea,0.693501
c_GarageArea,0.606081
c_TotalBsmtSF,0.558129
c_OpenPorchSF,0.506837
c_LotArea,0.388603
c_WoodDeckSF,0.378429
c_LotFrontage,0.366473
c_HalfBath,0.331280
c_EnclosedPorch,0.276808


In [ ]:
corr=corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False)

In [ ]:
cols_drop_low_corr=list(corr[corr["tgt_SalePrice"]<.1].index)

In [ ]:
cols_drop_low_corr

['c_ScreenPorch', 'c_BsmtFinSF2', 'c_MiscVal', 'c_3SsnPorch']

- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados

In [ ]:
X_train=X_train.drop(columns=cols_drop_low_corr)
X_test=X_test.drop(columns=cols_drop_low_corr)

### MULTICOLINEALIDAD

#### MULTICOLINEALIDAD CONTINUAS

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Función para multicolineaidad con continuas
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [ ]:
X_train.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)
X_test.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)

In [ ]:
for col in X_train.filter(like="c_").columns:
    X_train[col]=X_train[col].astype(float)

In [ ]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

In [ ]:
vif

,variables,VIF
0,c_LotFrontage,17.232131
1,c_LotArea,8.859464
2,c_BsmtFinSF1,41.124971
3,c_BsmtUnfSF,77.347971
4,c_TotalBsmtSF,177.450815
5,c_2ndFlrSF,8.769596
6,c_GrLivArea,66.221201
7,c_HalfBath,2.853188
8,c_BedroomAbvGr,22.313271
9,c_GarageArea,9.678761


In [ ]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False).head(50)

,variables,VIF
4,c_TotalBsmtSF,177.450815
0,c_LotFrontage,17.232131
6,c_GrLivArea,66.221201
3,c_BsmtUnfSF,77.347971
2,c_BsmtFinSF1,41.124971
8,c_BedroomAbvGr,22.313271


In [ ]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [ ]:
vif_feats

['c_TotalBsmtSF',
 'c_LotFrontage',
 'c_GrLivArea',
 'c_BsmtUnfSF',
 'c_BsmtFinSF1',
 'c_BedroomAbvGr']

In [ ]:
X_train.drop(columns=["c_TotalBsmtSF"],inplace=True)
X_test.drop(columns=["c_TotalBsmtSF"],inplace=True)

In [ ]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in ["tgt_SalePrice"]]])

In [ ]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotFrontage,17.185504
5,c_GrLivArea,65.037747
3,c_BsmtUnfSF,10.623386
7,c_BedroomAbvGr,21.765241


In [ ]:
vif_feats=["c_GrLivArea"]

In [ ]:
X_train.drop(columns=vif_feats,inplace=True)
X_test.drop(columns=vif_feats,inplace=True)

In [ ]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif[vif["VIF"]>7].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotFrontage,16.772564
1,c_LotArea,8.766217
7,c_GarageArea,8.721560
6,c_BedroomAbvGr,17.452252


In [ ]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [ ]:
vif_feats

['c_LotFrontage', 'c_BedroomAbvGr']

In [ ]:
X_train.drop(columns=["c_LotFrontage"],inplace=True)
X_test.drop(columns=["c_LotFrontage"],inplace=True)

In [ ]:
X_train.columns

Index(['v_MSSubClass', 'v_MSZoning', 'c_LotArea', 'v_Street', 'v_LotShape',
       'v_LandContour', 'v_LotConfig', 'v_LandSlope', 'v_Condition1',
       'v_Condition2', 'v_BldgType', 'v_HouseStyle', 'v_OverallQual',
       'v_OverallCond', 'v_RoofStyle', 'v_RoofMatl', 'v_MasVnrType',
       'v_ExterQual', 'v_ExterCond', 'v_Foundation', 'v_BsmtQual',
       'v_BsmtCond', 'v_BsmtExposure', 'v_BsmtFinType1', 'c_BsmtFinSF1',
       'v_BsmtFinType2', 'c_BsmtUnfSF', 'v_Heating', 'v_HeatingQC',
       'v_CentralAir', 'v_Electrical', 'c_2ndFlrSF', 'v_LowQualFinSF',
       'v_BsmtHalfBath', 'v_FullBath', 'c_HalfBath', 'c_BedroomAbvGr',
       'v_KitchenAbvGr', 'v_KitchenQual', 'v_Functional', 'v_Fireplaces',
       'v_GarageType', 'v_GarageFinish', 'c_GarageArea', 'v_GarageQual',
       'v_GarageCond', 'v_PavedDrive', 'c_WoodDeckSF', 'c_OpenPorchSF',
       'c_EnclosedPorch', 'tgt_SalePrice'],
      dtype='object')

In [ ]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif_feats=list(vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)["variables"][:10].values)

In [ ]:


vif[vif["VIF"]>7].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
6,c_GarageArea,8.437460
5,c_BedroomAbvGr,14.222746


In [ ]:
X_train.drop(columns=["c_BedroomAbvGr"],inplace=True)
X_test.drop(columns=["c_BedroomAbvGr"],inplace=True)

In [ ]:
#BsmtCond: Estado general del sótano
#ExterCond: Estado actual del material en el exterior

In [ ]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])
vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)

,variables,VIF
5,c_GarageArea,8.210211


In [ ]:
vif

,variables,VIF
0,c_LotArea,5.576486
1,c_BsmtFinSF1,3.350266
2,c_BsmtUnfSF,4.961821
3,c_2ndFlrSF,3.236757
4,c_HalfBath,2.812012
5,c_GarageArea,8.210211
6,c_WoodDeckSF,1.671976
7,c_OpenPorchSF,1.846390
8,c_EnclosedPorch,1.186067


#### MULTICOLINEALIDAD CATEGORICAS

In [ ]:
X_train

,v_MSSubClass,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice
0,20,otro,11096.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,8,5,gable,compshg,none,3,2,pconc,3.0,2.0,otro,glq,24.0,unf,1632.0,gasa,3,y,sbrkr,0.0,0,0,2,0.0,1,3,typ,0,attchd,rfn,826.0,2.0,2.0,y,0.0,146.0,0.0,249700
1,120,rl,7658.0,pave,reg,lvl,inside,gtl,norm,norm,otro,1story,9,5,hip,compshg,brkface,3,2,pconc,3.0,2.0,no,glq,456.0,unf,1296.0,gasa,3,y,sbrkr,0.0,0,0,2,0.0,1,3,typ,1,attchd,rfn,576.0,2.0,2.0,y,196.0,82.0,0.0,319900
2,20,rl,9600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,7,gable,compshg,none,2,2,cblock,3.0,2.0,no,otro,442.0,unf,625.0,gasa,2,y,sbrkr,0.0,0,0,2,0.0,1,3,otro,0,attchd,unf,436.0,2.0,2.0,y,290.0,0.0,0.0,128000
3,30,rl,11600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,5,gable,compshg,none,2,2,otro,1.0,2.0,no,unf,0.0,unf,700.0,gasa,3,y,sbrkr,0.0,0,0,1,0.0,1,1,typ,1,detchd,unf,252.0,2.0,1.0,y,0.0,0.0,67.0,137500
4,30,rl,5400.0,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,5,6,gable,compshg,none,2,2,otro,1.0,2.0,no,unf,0.0,unf,691.0,gasa,3,y,fusea,0.0,0,0,1,0.0,1,3,typ,0,detchd,unf,216.0,1.0,2.0,n,0.0,20.0,94.0,86000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,20,rl,11664.0,pave,reg,lvl,inside,gtl,otro,norm,1fam,1story,6,5,gable,compshg,brkface,2,2,cblock,2.0,1.0,no,otro,336.0,unf,746.0,gasa,2,y,sbrkr,0.0,0,0,1,0.0,1,2,typ,1,detchd,unf,240.0,2.0,2.0,y,0.0,130.0,0.0,119200
760,20,rl,7024.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,5,gable,compshg,none,2,2,pconc,3.0,3.0,no,alq,980.0,unf,110.0,gasa,3,y,sbrkr,0.0,0,0,1,1.0,1,2,typ,0,attchd,fin,450.0,2.0,2.0,y,0.0,49.0,0.0,157000
761,50,rl,12134.0,pave,otro,otro,inside,otro,norm,norm,1fam,1 5fin,8,7,gable,compshg,none,3,2,otro,3.0,2.0,otro,glq,427.0,unf,132.0,gasa,3,y,sbrkr,672.0,0,0,2,0.0,1,2,typ,0,otro,rfn,492.0,2.0,2.0,y,325.0,12.0,0.0,164000
762,190,rl,10615.0,pave,otro,otro,corner,otro,otro,otro,otro,2story,3,5,gable,compshg,none,2,2,cblock,1.0,2.0,otro,otro,440.0,unf,538.0,gasa,2,y,sbrkr,685.0,0,0,2,0.0,2,2,typ,0,otro,unf,420.0,1.0,1.0,y,0.0,74.0,0.0,95000


In [ ]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=False):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        #display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        #display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final


In [ ]:
v_feats=list(X_train.filter(like="v_").columns)

In [ ]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [ ]:
X_train["v_BldgType"].value_counts()

1fam    639
otro    125
Name: v_BldgType, dtype: int64

In [ ]:
X_train["v_OverallQual"].value_counts()

5     222
6     190
7     180
8      76
4      67
9      12
3      11
2       3
10      3
Name: v_OverallQual, dtype: int64

In [ ]:
X_train["v_OverallCond"].value_counts()

5    439
6    128
7    112
8     38
4     26
3     10
9      9
2      2
Name: v_OverallCond, dtype: int64

In [ ]:
X_train[["v_OverallQual","v_OverallCond"]].head(10)

,v_OverallQual,v_OverallCond
0,8,5
1,9,5
2,4,7
3,4,5
4,5,6
5,7,4
6,7,5
7,7,5
8,5,5
9,5,5


In [ ]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSSubClass,1.822921e+01,1,4.269567,18.229205
v_MSZoning,2.218767e+00,2,1.220472,1.489553
v_Street,1.322104e+00,1,1.149828,1.322104
v_LotShape,1.616980e+00,1,1.271605,1.616980
v_LandContour,1.032380e+00,1,1.016061,1.032380
v_LotConfig,1.362874e+00,2,1.080473,1.167422
v_LandSlope,9.264444e-01,1,0.962520,0.926444
v_Condition1,1.081945e+00,1,1.040166,1.081945
v_Condition2,1.727949e+00,1,1.314515,1.727949


In [ ]:
X_train=X_train.drop(columns=["v_MSSubClass","v_OverallQual"])
X_test=X_test.drop(columns=["v_MSSubClass","v_OverallQual"])

In [ ]:
v_feats=list(X_train.filter(like="v_").columns)

In [ ]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [ ]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSZoning,2.742581e+00,2,1.286885,1.656074
v_Street,1.511939e+00,1,1.229609,1.511939
v_LotShape,1.750340e+00,1,1.323004,1.750340
v_LandContour,1.281185e+00,1,1.131895,1.281185
v_LotConfig,1.424896e+00,2,1.092561,1.193690
v_LandSlope,1.602087e+00,1,1.265736,1.602087
v_Condition1,1.219854e+00,1,1.104470,1.219854
v_Condition2,2.424682e+00,1,1.557139,2.424682
v_BldgType,2.460389e+00,1,1.568563,2.460389


In [ ]:
X_train

,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice
0,otro,11096.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,3,2,pconc,3.0,2.0,otro,glq,24.0,unf,1632.0,gasa,3,y,sbrkr,0.0,0,0,2,0.0,1,3,typ,0,attchd,rfn,826.0,2.0,2.0,y,0.0,146.0,0.0,249700
1,rl,7658.0,pave,reg,lvl,inside,gtl,norm,norm,otro,1story,5,hip,compshg,brkface,3,2,pconc,3.0,2.0,no,glq,456.0,unf,1296.0,gasa,3,y,sbrkr,0.0,0,0,2,0.0,1,3,typ,1,attchd,rfn,576.0,2.0,2.0,y,196.0,82.0,0.0,319900
2,rl,9600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,7,gable,compshg,none,2,2,cblock,3.0,2.0,no,otro,442.0,unf,625.0,gasa,2,y,sbrkr,0.0,0,0,2,0.0,1,3,otro,0,attchd,unf,436.0,2.0,2.0,y,290.0,0.0,0.0,128000
3,rl,11600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,2,2,otro,1.0,2.0,no,unf,0.0,unf,700.0,gasa,3,y,sbrkr,0.0,0,0,1,0.0,1,1,typ,1,detchd,unf,252.0,2.0,1.0,y,0.0,0.0,67.0,137500
4,rl,5400.0,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,6,gable,compshg,none,2,2,otro,1.0,2.0,no,unf,0.0,unf,691.0,gasa,3,y,fusea,0.0,0,0,1,0.0,1,3,typ,0,detchd,unf,216.0,1.0,2.0,n,0.0,20.0,94.0,86000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,rl,11664.0,pave,reg,lvl,inside,gtl,otro,norm,1fam,1story,5,gable,compshg,brkface,2,2,cblock,2.0,1.0,no,otro,336.0,unf,746.0,gasa,2,y,sbrkr,0.0,0,0,1,0.0,1,2,typ,1,detchd,unf,240.0,2.0,2.0,y,0.0,130.0,0.0,119200
760,rl,7024.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,2,2,pconc,3.0,3.0,no,alq,980.0,unf,110.0,gasa,3,y,sbrkr,0.0,0,0,1,1.0,1,2,typ,0,attchd,fin,450.0,2.0,2.0,y,0.0,49.0,0.0,157000
761,rl,12134.0,pave,otro,otro,inside,otro,norm,norm,1fam,1 5fin,7,gable,compshg,none,3,2,otro,3.0,2.0,otro,glq,427.0,unf,132.0,gasa,3,y,sbrkr,672.0,0,0,2,0.0,1,2,typ,0,otro,rfn,492.0,2.0,2.0,y,325.0,12.0,0.0,164000
762,rl,10615.0,pave,otro,otro,corner,otro,otro,otro,otro,2story,5,gable,compshg,none,2,2,cblock,1.0,2.0,otro,otro,440.0,unf,538.0,gasa,2,y,sbrkr,685.0,0,0,2,0.0,2,2,typ,0,otro,unf,420.0,1.0,1.0,y,0.0,74.0,0.0,95000


In [ ]:
X_train["v_LowQualFinSF"].value_counts()

0      753
80       2
360      1
514      1
156      1
390      1
513      1
232      1
473      1
144      1
234      1
Name: v_LowQualFinSF, dtype: int64

In [ ]:
##CREAR LAS VARIABLES DUMMY

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
dummy_vars=["v_MSZoning","v_Street","v_LotShape","v_LandContour","v_LotConfig","v_LandSlope","v_Condition1","v_Condition2","v_BldgType","v_HouseStyle",
"v_RoofStyle","v_MasVnrType","v_Foundation","v_BsmtExposure","v_BsmtFinType1","v_BsmtFinType2","v_Heating","v_CentralAir",
"v_Electrical","v_Functional","v_GarageType","v_GarageFinish","v_PavedDrive","v_RoofMatl","v_OverallCond"]

In [ ]:
X_train.shape

(764, 48)

In [ ]:
X_test.shape

(192, 48)

In [ ]:
X_test

,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice
0,rl,8248,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,7,gable,compshg,none,2,2,otro,2.0,2.0,no,unf,0,unf,686,otro,3,y,sbrkr,564,0,1,1,1,1,1,typ,0,detchd,unf,280,2.0,2.0,p,207,0,96,116000
1,rm,8960,pave,reg,lvl,inside,gtl,otro,norm,1fam,1story,6,gable,compshg,none,2,2,cblock,2.0,2.0,no,unf,0,unf,1008,gasa,3,y,fusea,0,0,0,1,0,1,2,typ,1,detchd,unf,360,2.0,2.0,y,0,0,130,115000
2,otro,7136,pave,otro,otro,inside,gtl,norm,norm,otro,2story,6,gable,compshg,brkface,2,2,cblock,3.0,2.0,no,otro,484,unf,495,gasa,2,n,otro,979,0,0,2,0,2,2,typ,0,attchd,unf,492,2.0,2.0,y,0,0,0,145000
3,otro,9000,pave,reg,lvl,corner,gtl,norm,norm,1fam,otro,3,gable,compshg,none,2,2,otro,1.0,1.0,no,unf,0,unf,784,gasa,2,n,fusea,0,0,0,1,0,1,2,typ,0,detchd,unf,360,1.0,1.0,n,0,0,91,76000
4,otro,8125,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,5,gable,compshg,none,3,2,pconc,3.0,2.0,no,unf,0,unf,756,gasa,3,y,sbrkr,797,0,0,2,1,1,3,typ,0,attchd,rfn,615,2.0,2.0,y,0,45,0,186500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,otro,8640,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,otro,3,2,pconc,3.0,2.0,otro,glq,936,unf,364,gasa,3,y,sbrkr,0,0,0,2,0,1,3,typ,0,attchd,rfn,552,2.0,2.0,y,135,112,0,229456
188,rl,8400,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,brkface,3,2,pconc,3.0,2.0,otro,glq,712,unf,761,gasa,3,y,sbrkr,0,0,0,2,0,1,3,typ,0,attchd,rfn,606,2.0,2.0,y,0,35,0,213000
189,rl,7590,pave,reg,lvl,inside,gtl,otro,norm,1fam,2story,5,gable,compshg,brkface,2,2,cblock,2.0,2.0,no,otro,512,unf,148,gasa,2,y,sbrkr,688,0,0,1,1,1,2,typ,1,attchd,rfn,453,2.0,2.0,y,188,108,0,155000
190,rm,3675,pave,reg,lvl,inside,gtl,norm,norm,otro,otro,5,gable,compshg,brkface,2,2,pconc,3.0,2.0,otro,glq,547,unf,0,gasa,3,y,sbrkr,0,0,0,2,0,1,2,typ,0,otro,fin,525,2.0,2.0,y,0,44,0,140000


In [ ]:
one=OneHotEncoder(drop="first",handle_unknown ="error")
#drop="first"

one.fit(X_train[dummy_vars])

replace_name=dict(zip(list(map(lambda x:f"x{str(x)}_",range(len(dummy_vars)))),dummy_vars))

features=one.get_feature_names()

features=list(map(lambda x:multiple_replace(x, replace_name),features))

X_train[features]=pd.DataFrame(one.transform(X_train[dummy_vars]).toarray(),columns=features)
X_test[features]=pd.DataFrame(one.transform(X_test[dummy_vars]).toarray(),columns=features)

X_train.drop(columns=dummy_vars,inplace=True)
X_test.drop(columns=dummy_vars,inplace=True)

In [ ]:
X_train

,c_LotArea,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,c_BsmtFinSF1,c_BsmtUnfSF,v_HeatingQC,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,c_GarageArea,v_GarageQual,v_GarageCond,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,11096.0,3,2,3.0,2.0,24.0,1632.0,3,0.0,0,0,2,0.0,1,3,0,826.0,2.0,2.0,0.0,146.0,0.0,249700,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,7658.0,3,2,3.0,2.0,456.0,1296.0,3,0.0,0,0,2,0.0,1,3,1,576.0,2.0,2.0,196.0,82.0,0.0,319900,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,9600.0,2,2,3.0,2.0,442.0,625.0,2,0.0,0,0,2,0.0,1,3,0,436.0,2.0,2.0,290.0,0.0,0.0,128000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,11600.0,2,2,1.0,2.0,0.0,700.0,3,0.0,0,0,1,0.0,1,1,1,252.0,2.0,1.0,0.0,0.0,67.0,137500,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5400.0,2,2,1.0,2.0,0.0,691.0,3,0.0,0,0,1,0.0,1,3,0,216.0,1.0,2.0,0.0,20.0,94.0,86000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,11664.0,2,2,2.0,1.0,336.0,746.0,2,0.0,0,0,1,0.0,1,2,1,240.0,2.0,2.0,0.0,130.0,0.0,119200,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
760,7024.0,2,2,3.0,3.0,980.0,110.0,3,0.0,0,0,1,1.0,1,2,0,450.0,2.0,2.0,0.0,49.0,0.0,157000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,12134.0,3,2,3.0,2.0,427.0,132.0,3,672.0,0,0,2,0.0,1,2,0,492.0,2.0,2.0,325.0,12.0,0.0,164000,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
762,10615.0,2,2,1.0,2.0,440.0,538.0,2,685.0,0,0,2,0.0,2,2,0,420.0,1.0,1.0,0.0,74.0,0.0,95000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

**Necesitamos normalizar los datos antes del análisis de componentes principales (PCA)**
- El PCA calcula una nueva proyección de su conjunto de datos. Y el nuevo eje se basa en la desviación estándar de sus variables. Por tanto, una variable con una desviación estándar alta tendrá un peso mayor para el cálculo del eje que una variable con una desviación estándar baja. Si normaliza sus datos, todas las variables tienen la misma desviación estándar, por lo que todas las variables tienen el mismo peso y su PCA calcula el eje relevante.

In [ ]:
X_train.filter(like="c_")

,c_LotArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_HalfBath,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch
0,11096.0,24.0,1632.0,0.0,0.0,826.0,0.0,146.0,0.0
1,7658.0,456.0,1296.0,0.0,0.0,576.0,196.0,82.0,0.0
2,9600.0,442.0,625.0,0.0,0.0,436.0,290.0,0.0,0.0
3,11600.0,0.0,700.0,0.0,0.0,252.0,0.0,0.0,67.0
4,5400.0,0.0,691.0,0.0,0.0,216.0,0.0,20.0,94.0
...,...,...,...,...,...,...,...,...,...
759,11664.0,336.0,746.0,0.0,0.0,240.0,0.0,130.0,0.0
760,7024.0,980.0,110.0,0.0,1.0,450.0,0.0,49.0,0.0
761,12134.0,427.0,132.0,672.0,0.0,492.0,325.0,12.0,0.0
762,10615.0,440.0,538.0,685.0,0.0,420.0,0.0,74.0,0.0


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train.filter(like="c_"))
df_train_sc=pd.DataFrame(scaler.transform(X_train.filter(like="c_")))
df_test_sc=pd.DataFrame(scaler.transform(X_test.filter(like="c_")))

In [ ]:
df_train_sc

,0,1,2,3,4,5,6,7,8
0,0.463638,-0.906142,2.393902,-0.778156,-0.689132,1.955486,-0.717520,2.103465,-0.358120
1,-0.368221,0.229449,1.604675,-0.778156,-0.689132,0.652897,1.212814,0.852784,-0.358120
2,0.101666,0.192647,0.028571,-0.778156,-0.689132,-0.076553,2.138586,-0.749652,-0.358120
3,0.585586,-0.969230,0.204738,-0.778156,-0.689132,-1.035258,-0.717520,-0.749652,0.881665
4,-0.914568,-0.969230,0.183598,-0.778156,-0.689132,-1.222831,-0.717520,-0.358814,1.381280
...,...,...,...,...,...,...,...,...,...
759,0.601072,-0.085993,0.312786,-0.778156,-0.689132,-1.097782,-0.717520,1.790795,-0.358120
760,-0.521624,1.606878,-1.181107,-0.778156,1.451100,-0.003608,-0.717520,0.207901,-0.358120
761,0.714793,0.153217,-1.129431,0.952748,-0.689132,0.215227,2.483289,-0.515149,-0.358120
762,0.347256,0.187390,-0.175782,0.986233,-0.689132,-0.159918,-0.717520,0.696449,-0.358120


In [ ]:
df_train_sc.describe()

,0,1,2,3,4,5,6,7,8
count,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02
mean,-8.370268e-17,5.580179e-17,1.209039e-16,1.860060e-17,6.045193e-17,1.162537e-16,-6.975223e-17,2.790089e-17,3.022597e-17
std,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00
min,-1.906605e+00,-9.692301e-01,-1.439484e+00,-7.781565e-01,-6.891324e-01,-2.348267e+00,-7.175200e-01,-7.496517e-01,-3.581204e-01
25%,-4.790390e-01,-9.692301e-01,-7.847316e-01,-7.781565e-01,-6.891324e-01,-7.434780e-01,-7.175200e-01,-7.496517e-01,-3.581204e-01
50%,-3.165436e-02,-1.267376e-01,-1.405490e-01,-7.781565e-01,-6.891324e-01,5.370621e-02,-7.175200e-01,-3.588138e-01,-3.581204e-01
75%,3.843359e-01,7.302127e-01,5.834958e-01,9.843008e-01,1.451100e+00,6.099116e-01,7.006845e-01,4.228621e-01,-3.581204e-01
max,1.490020e+01,2.768755e+00,3.262990e+00,2.750622e+00,1.451100e+00,3.138236e+00,5.536368e+00,5.914135e+00,5.748285e+00


In [ ]:
for n_compo in range(1,10):
    print(f"N componentes : {n_compo}")
    pca = PCA(n_components=n_compo)
    pca.fit(df_train_sc)
    print(sum(pca.explained_variance_ratio_))
    print("\n")

N componentes : 1
0.21884242370528356


N componentes : 2
0.4057855522550133


N componentes : 3
0.5735874968385434


N componentes : 4
0.6800909776468353


N componentes : 5
0.7757140912555798


N componentes : 6
0.8645888022735909


N componentes : 7
0.9391150139776069


N componentes : 8
0.9756902243755323


N componentes : 9
1.0




In [ ]:
pca = PCA(n_components=7)
pca.fit(df_train_sc)

PCA(n_components=7)

In [ ]:
X_train_pca=pd.DataFrame(pca.transform(df_train_sc))
X_test_pca=pd.DataFrame(pca.transform(df_test_sc))

In [ ]:
#Porcentaje de varianza explicada por cada uno de los componentes seleccionados.
pca.explained_variance_ratio_

array([0.21884242, 0.18694313, 0.16780194, 0.10650348, 0.09562311,
       0.08887471, 0.07452621])

In [ ]:
sum(pca.explained_variance_ratio_)

0.939115013977607

In [ ]:
#Para visualización creamos dos o tres componentes para tener una idea de como se ve nuestra información continua
#Tomando en cuenta la varianza explicada (algunas veces no es tan representativo)
pca = PCA(n_components=2)
dimention_2=pca.fit_transform(df_train_sc)
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter(dimention_2, x=0, y=1,title=f'Total de Varianza Explicada: {total_var:.2f}%',
                 labels={'0': 'Componente Principal 1', '1': 'Componente Principal 2'})
fig.show()

In [ ]:
pca = PCA(n_components=3)
dimention_3=pca.fit_transform(df_train_sc)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    dimention_3, x=0, y=1, z=2, 
    title=f'Total de Varianza Explicada: {total_var:.2f}%',
    labels={'0': 'CP 1', '1': 'CP 2', '2': 'CP 3'}
)
fig.show()

### K- Best

Score function:
- For regression: f_regression, mutual_info_regression
- For classification: chi2, f_classif, mutual_info_classif

**f_regression**
- Pruebas de regresión lineal univariante.

- Modelo lineal para probar el efecto individual de cada uno de los muchos regresores. Esta es una función de puntuación que se utilizará en un procedimiento de selección de características

- Esto se hace en 2 pasos:

- Se calcula la correlación entre cada regresor y el objetivo, es decir, ((X [:, i] - mean (X [:, i])) * (y - mean_y)) / (std (X [:, i] ) * estándar (y)).

- Se convierte en una puntuación F y luego en un valor p.

- mutual_info_classif : para discretas y continuas
- f_regressión : solo continuas

https://towardsdatascience.com/fisher-test-for-regression-analysis-1e1687867259

** Estime la información mutua para una variable objetivo discreta.

** La información mutua (MI) [1] entre dos variables aleatorias es un valor no negativo, que mide la dependencia entre las variables. Es igual a cero si y solo si dos variables aleatorias son independientes, y los valores más altos significan una mayor dependencia.

** La función se basa en métodos no paramétricos basados ​​en la estimación de la entropía a partir de distancias de los vecinos más cercanos k.

In [ ]:
X=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_")
y=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

In [ ]:
from sklearn.feature_selection import SelectKBest,mutual_info_classif 

In [ ]:
kb = SelectKBest(mutual_info_classif , k=5)

In [ ]:
X

,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_HalfBath,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal
0,6240.0,0.0,0.0,816.0,0.0,0.0,528.0,112.0,0.0,0.0,400.0
1,7742.0,0.0,763.0,192.0,0.0,0.0,386.0,0.0,0.0,0.0,0.0
2,9600.0,0.0,442.0,625.0,0.0,0.0,436.0,290.0,0.0,0.0,0.0
3,11600.0,0.0,0.0,700.0,0.0,0.0,252.0,0.0,0.0,67.0,0.0
4,10206.0,0.0,0.0,0.0,0.0,0.0,528.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
759,21286.0,0.0,0.0,720.0,551.0,0.0,312.0,0.0,0.0,108.0,0.0
760,8834.0,216.0,1170.0,292.0,762.0,1.0,738.0,184.0,0.0,0.0,0.0
761,7094.0,0.0,180.0,340.0,0.0,0.0,384.0,0.0,0.0,0.0,0.0
762,2645.0,456.0,0.0,776.0,677.0,1.0,492.0,206.0,0.0,0.0,0.0


In [ ]:
X_new=kb.fit_transform(X, y)

In [ ]:
cols=X.columns[kb.get_support()]

In [ ]:
pd.DataFrame(X_new,columns=cols)

,c_LotArea,c_MasVnrArea,c_HalfBath,c_GarageArea,c_OpenPorchSF
0,6240.0,0.0,0.0,528.0,0.0
1,7742.0,0.0,0.0,386.0,0.0
2,9600.0,0.0,0.0,436.0,0.0
3,11600.0,0.0,0.0,252.0,0.0
4,10206.0,0.0,0.0,528.0,0.0
...,...,...,...,...,...
759,21286.0,0.0,0.0,312.0,0.0
760,8834.0,216.0,1.0,738.0,0.0
761,7094.0,0.0,0.0,384.0,0.0
762,2645.0,456.0,1.0,492.0,0.0


In [ ]:
scores=pd.DataFrame(kb.scores_)
scores["Variable"]=X.columns
scores.rename(columns={0:"Score"},inplace=True)

In [ ]:
scores=scores.sort_values(by="Score",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [ ]:
scores

,Score,Variable
0,0.335697,c_HalfBath
1,0.249196,c_GarageArea
2,0.195370,c_OpenPorchSF
3,0.164919,c_LotArea
4,0.117103,c_MasVnrArea
5,0.074608,c_WoodDeckSF
6,0.065104,c_MiscVal
7,0.055629,c_2ndFlrSF
8,0.044195,c_BsmtUnfSF
9,0.030172,c_BsmtFinSF1


In [ ]:
X[list(scores[:20]["Variable"].values)][cols]

,c_LotArea,c_MasVnrArea,c_HalfBath,c_GarageArea,c_OpenPorchSF
0,6240.0,0.0,0.0,528.0,0.0
1,7742.0,0.0,0.0,386.0,0.0
2,9600.0,0.0,0.0,436.0,0.0
3,11600.0,0.0,0.0,252.0,0.0
4,10206.0,0.0,0.0,528.0,0.0
...,...,...,...,...,...
759,21286.0,0.0,0.0,312.0,0.0
760,8834.0,216.0,1.0,738.0,0.0
761,7094.0,0.0,0.0,384.0,0.0
762,2645.0,456.0,1.0,492.0,0.0


## VarClus

- Varclus es un buen algoritmo de reducción de dimensiones. Aquí hay una breve descripción:
- Se elige un grupo para dividir.
- El grupo elegido se divide en dos grupos al encontrar los dos primeros componentes principales, realizar una rotación ortoblicua y asignar cada variable al componente girado con el que tiene la correlación al cuadrado más alto.
- Las variables se reasignan iterativamente a los conglomerados para maximizar la varianza contabilizada por los componentes del conglomerado

In [ ]:
from varclushi import VarClusHi

In [ ]:
vc = VarClusHi(X,maxeigval2=.7)

In [ ]:
vc.varclus()

In [ ]:
vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_EnclosedPorch,1.000000,0.033443,0.000000
1,1,c_BsmtFinSF1,0.786901,0.039508,0.221865
2,1,c_BsmtUnfSF,0.786901,0.037686,0.221445
3,2,c_2ndFlrSF,0.821109,0.031683,0.184744
4,2,c_HalfBath,0.821109,0.037034,0.185771
5,3,c_WoodDeckSF,1.000000,0.043775,0.000000
6,4,c_MiscVal,1.000000,0.004830,0.000000
7,5,c_GarageArea,1.000000,0.064245,0.000000
8,6,c_LotArea,1.000000,0.049686,0.000000
9,7,c_OpenPorchSF,1.000000,0.062331,0.000000


In [ ]:
vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_EnclosedPorch,1.000000,0.033443,0.000000
2,1,c_BsmtUnfSF,0.786901,0.037686,0.221445
3,2,c_2ndFlrSF,0.821109,0.031683,0.184744
5,3,c_WoodDeckSF,1.000000,0.043775,0.000000
6,4,c_MiscVal,1.000000,0.004830,0.000000
7,5,c_GarageArea,1.000000,0.064245,0.000000
8,6,c_LotArea,1.000000,0.049686,0.000000
9,7,c_OpenPorchSF,1.000000,0.062331,0.000000
10,8,c_MasVnrArea,1.000000,0.064245,0.000000


In [ ]:
columns=list(vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")["Variable"].values)

In [ ]:
columns

['c_EnclosedPorch',
 'c_BsmtUnfSF',
 'c_2ndFlrSF',
 'c_WoodDeckSF',
 'c_MiscVal',
 'c_GarageArea',
 'c_LotArea',
 'c_OpenPorchSF',
 'c_MasVnrArea']

## Arboles de decision

In [ ]:
x_train=X_train[[x for x in X_train if x!="tgt_SalePrice"]]
y_train=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

x_test=X_test[[x for x in X_test if x!="tgt_SalePrice"]]
y_test=X_test[[x for x in X_test if x=="tgt_SalePrice"]]

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:

model = DecisionTreeRegressor()

model.fit(x_train, y_train)

importance = model.feature_importances_

In [ ]:
pred_test=model.predict(x_test)
pred_train=model.predict(x_train)

In [ ]:
r2_score(y_test,pred_test)

0.6140452532473888

In [ ]:
r2_score(y_train,pred_train)

0.9999861620309304

In [ ]:
importance

array([6.69053466e-02, 4.33268460e-01, 8.36428324e-04, 2.56931339e-02,
       2.39087874e-04, 7.46336773e-02, 3.29868204e-02, 1.64186511e-03,
       5.10341461e-02, 4.61272019e-08, 1.15365836e-03, 1.70152572e-02,
       4.14195964e-03, 6.81591034e-04, 3.33235983e-04, 2.63857276e-02,
       1.20733445e-01, 1.54030883e-04, 4.12069670e-06, 9.67315081e-03,
       1.41330542e-02, 4.55659166e-03, 3.17428284e-04, 2.92075162e-03,
       0.00000000e+00, 2.58376731e-03, 2.55885821e-03, 6.42586209e-04,
       2.21771425e-04, 0.00000000e+00, 4.68056792e-04, 2.24733573e-05,
       5.61743274e-04, 6.52819477e-04, 4.38399509e-04, 7.62129005e-06,
       6.34715260e-03, 0.00000000e+00, 4.76440031e-03, 3.19581162e-03,
       2.33373412e-04, 9.65177890e-04, 2.45222361e-03, 3.97646878e-04,
       4.88787751e-04, 2.94440560e-03, 5.70815032e-04, 4.53921188e-08,
       9.50662721e-03, 5.03164907e-04, 7.50934334e-04, 1.06827316e-03,
       3.71394048e-03, 1.05402501e-04, 9.49754297e-04, 6.05250520e-04,
      

In [ ]:
scores=pd.DataFrame()
scores["gini_index"]=importance
scores["feature"]=x_train.columns

In [ ]:
scores=scores.sort_values(by="gini_index",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [ ]:
feats=list(scores[:10]["feature"].values)

In [ ]:
scores

,gini_index,feature
0,4.332685e-01,v_ExterQual
1,1.207334e-01,c_GarageArea
2,7.463368e-02,c_BsmtFinSF1
3,6.690535e-02,c_LotArea
4,5.465085e-02,v_GarageFinish_unf
...,...,...
62,4.612720e-08,v_LowQualFinSF
63,4.539212e-08,v_Heating_otro
64,0.000000e+00,v_RoofStyle_otro
65,0.000000e+00,v_LandSlope_otro
